# Search Tool Greek Hotels: TwitterAPI

----
Libraries:

In [9]:
import re
import numpy as np
import pandas as pd
import json 
from batoomer.twitter_nodes.search_engine import TwitterSearchEngine
from googleapiclient.errors import HttpError
from tqdm.notebook import tqdm
import time
import tweepy

---
Load Google Credentials and initialize GoogleSearchEngine

In [3]:
keys = []
with open('../../../../twitter_credentials.json', 'r') as f:
    keys = json.load(f)

se = TwitterSearchEngine(twitter_credentials=keys, )

---
## Dataset

In [4]:
data = pd.read_csv('../../../../datasets/Hotels/search-tool/greek-hotels-2018-search.csv', delimiter=';')
data.replace(np.nan, '', inplace=True)
data.head(5)

,Hotel City,Hotelname,Twitter Handle,Website
0,Faliraki,Hotel Calypso Palace,,https://www.calypsopalace.gr/
1,Platanes,Nefeli Hotel,,https://nefeli-hotel.gr/
2,Kallithea,Hotel Kalithea Horizon Royal,,https://www.kalitheahorizon.gr/
3,Archangelos,Porto Angeli Beach Resort Hotel,portoangeli,https://www.portoangeli.com/
4,Faliraki,Hotel Calypso Beach,,http://www.calypsohotel.gr/


---
## Search: Hotel

In [10]:
queries = list(data['Hotelname'])
results = pd.DataFrame()

for query in tqdm(queries):
    try:
        se.search(query=query, count=10)
        result = se.get_results()
        results= results.append(result)
    except Exception as err:
        print(err)

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
results.replace(np.nan, 0, inplace=True)

In [12]:
results = (results
           .replace(np.nan, '')
           .rename(columns={'Query':'Hotelname'})
           .reset_index()
           .drop('index', axis=1))
results = pd.merge(data, results, on=results.index, how='outer').drop(['key_0', 'Hotelname_y'], axis =1).rename(columns={'Hotelname_x':'Hotelname'})
results.head(5)

,Hotel City,Hotelname,Twitter Handle,Website,Result 1,Result 2,Result 3,Result 4,Result 5,Result 6,...,Result 11,Result 12,Result 13,Result 14,Result 15,Result 16,Result 17,Result 18,Result 19,Result 20
0,Faliraki,Hotel Calypso Palace,,https://www.calypsopalace.gr/,1308314363086897152,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Platanes,Nefeli Hotel,,https://nefeli-hotel.gr/,364266196,388472346,747720708726349824,2440309615,1445935388,383129663,...,0,0,0,0,0,0,0,0,0,0
2,Kallithea,Hotel Kalithea Horizon Royal,,https://www.kalitheahorizon.gr/,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Archangelos,Porto Angeli Beach Resort Hotel,portoangeli,https://www.portoangeli.com/,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Faliraki,Hotel Calypso Beach,,http://www.calypsohotel.gr/,722657046122913793,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
results.to_csv('Twitter-Search-Hotelname-Hotels.csv', index=False)